# 

In [2]:
from pyln.client import Gossmap, GossmapNodeId, GossmapNode, GossmapStats

gossmap = Gossmap('gossip_stores/nov_17.gossip_store')

In [3]:
channel = gossmap.get_channel('803775x1091x1')
channel.satoshis

2000000

In [4]:
channel.features

0

In [5]:
channel.half_channels[0].source

GossmapNode[022bd0aa893db4ac890e457cca8c83f112518d6941bf9153dab4bf904620503a78, "lnbits.com                      "]

In [6]:
channel.half_channels[0].destination.node_id

GossmapNodeId[02d503276074ce65fd3aefd220e2fdbd272a9fc0bd23c32f529a53a9c98022abe4]

In [7]:
gossmap.channels

{ShortChannelId[761920x1276x0]: GossmapChannel[761920x1276x0],
 ShortChannelId[702790x539x1]: GossmapChannel[702790x539x1],
 ShortChannelId[721770x2518x0]: GossmapChannel[721770x2518x0],
 ShortChannelId[785923x1933x10]: GossmapChannel[785923x1933x10],
 ShortChannelId[793907x4235x1]: GossmapChannel[793907x4235x1],
 ShortChannelId[771083x1469x1]: GossmapChannel[771083x1469x1],
 ShortChannelId[768170x732x0]: GossmapChannel[768170x732x0],
 ShortChannelId[770873x2090x0]: GossmapChannel[770873x2090x0],
 ShortChannelId[799842x1310x5]: GossmapChannel[799842x1310x5],
 ShortChannelId[785910x1149x12]: GossmapChannel[785910x1149x12],
 ShortChannelId[799842x1310x3]: GossmapChannel[799842x1310x3],
 ShortChannelId[769299x344x27]: GossmapChannel[769299x344x27],
 ShortChannelId[769306x236x0]: GossmapChannel[769306x236x0],
 ShortChannelId[769220x1924x1]: GossmapChannel[769220x1924x1],
 ShortChannelId[785923x1933x15]: GossmapChannel[785923x1933x15],
 ShortChannelId[692080x1833x0]: GossmapChannel[692080x1

In [8]:
gossmap.get_neighbors_hc(destination=channel.half_channels[0].destination.node_id)

{GossmapHalfchannel[803773x2518x0/0],
 GossmapHalfchannel[803773x2518x2/1],
 GossmapHalfchannel[803775x1091x0/1],
 GossmapHalfchannel[803775x1091x1/0],
 GossmapHalfchannel[804114x1166x0/1],
 GossmapHalfchannel[805073x2021x1/1],
 GossmapHalfchannel[805073x2021x2/0],
 GossmapHalfchannel[805073x2021x3/0],
 GossmapHalfchannel[808592x2287x6/1],
 GossmapHalfchannel[809266x2756x0/0],
 GossmapHalfchannel[809277x524x0/1]}

In [9]:
gossmap.get_neighbors_hc(source=channel.half_channels[0].destination.node_id)

{GossmapHalfchannel[803773x2518x0/1],
 GossmapHalfchannel[803773x2518x2/0],
 GossmapHalfchannel[803775x1091x0/0],
 GossmapHalfchannel[803775x1091x1/1],
 GossmapHalfchannel[804114x1166x0/0],
 GossmapHalfchannel[805073x2021x1/0],
 GossmapHalfchannel[805073x2021x2/1],
 GossmapHalfchannel[805073x2021x3/1],
 GossmapHalfchannel[808592x2287x6/0],
 GossmapHalfchannel[809266x2756x0/1],
 GossmapHalfchannel[809277x524x0/0]}

In [10]:
def find_neighbors(gossmap, node, depth=0):
    neighbors = gossmap.get_neighbors(node, depth=depth)
    return neighbors

In [28]:

my_source = GossmapNodeId("0235acd5e943cbe5387815a8fb15ebc109b26556f82fa28cfcb0c2a3fda7da80fa")
#my_source = GossmapNodeId("0202e7727ee46d943a44c7bf0dce14220a839630dee67d03446bf935178b55a0df")
my_destination = GossmapNodeId("02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26")
#my_destination = GossmapNodeId("03cabeaa1223b83d56954e944e83493d20c6b2c9b45e47b62a899ab9d7d31cb56d")


In [29]:
path = find_route(gossmap, my_source, my_destination) 

In [30]:
# challenge: what channels should i use to get 
# from 0202e7727ee46d943a44c7bf0dce14220a839630dee67d03446bf935178b55a0df
# to   02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26
MAX_SEARCH_DEPTH = 1000


def find_route(gossmap, source, destination):
    distance, path_node = find_distance(gossmap, source, destination)
    return find_route_inner(gossmap, source, destination, distance=distance, path_node=path_node)
    

def find_distance(gossmap, source, destination):
    if are_neighbors(gossmap, source, destination):
        return 1, None
        
    overlap = {}
    left_depth = 1
    right_depth = 1

    while left_depth < MAX_SEARCH_DEPTH:
        overlap = get_overlap(gossmap, source, left_depth, right_depth, destination)
        if bool(overlap):
            break
        if left_depth == right_depth:
            right_depth += 1
        else: 
            left_depth += 1
    
    distance = left_depth + right_depth
    path_node = list(overlap)[0]
    
    return distance, path_node
    

def get_overlap(gossmap, source, left_depth, right_depth, destination):
        neighbors_source = find_neighbors(gossmap, source, depth=left_depth)
        neighbors_destination = find_neighbors(gossmap, destination, depth=right_depth)
        overlap = neighbors_source.intersection(neighbors_destination)
        return overlap


def find_route_inner(gossmap, source, destination, distance, path_node):
    
    if distance == 1:
        return [source, destination]
    if distance == 2:
        return [source, path_node, destination]
    
    distance_left, node_left = find_distance(gossmap, source, path_node)
    distance_right, node_right = find_distance(gossmap, path_node, destination)
    
    left_path = find_route_inner(gossmap, source, path_node, distance_left, node_left)
    right_path = find_route_inner(gossmap, path_node, destination, distance_right, node_right)
    
    return left_path[:-1] + right_path
    
    
    
def are_neighbors(gossmap, source, destination):
    neighbors_to_source = find_neighbors(gossmap, source, depth=1)
    return destination in neighbors_to_source

path = find_route(gossmap, my_source, my_destination)
path

[GossmapNodeId[0235acd5e943cbe5387815a8fb15ebc109b26556f82fa28cfcb0c2a3fda7da80fa],
 GossmapNode[02f1a8c87607f415c8f22c00593002775941dea48869ce23096af27b0cfdcc0b69, "Kraken 🐙⚡                  "],
 GossmapNodeId[02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26]]

In [31]:
path = find_route(gossmap, my_source, my_destination)

In [48]:
path

[GossmapNodeId[0235acd5e943cbe5387815a8fb15ebc109b26556f82fa28cfcb0c2a3fda7da80fa],
 GossmapNode[02f1a8c87607f415c8f22c00593002775941dea48869ce23096af27b0cfdcc0b69, "Kraken 🐙⚡                  "],
 GossmapNodeId[02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26]]

In [49]:
payment_amount = 10000 * 1000 # in millisatoshis

In [55]:
node_one = gossmap.get_node(path[0])
node_two = path[1]
node_three = gossmap.get_node(path[2])

def find_channel(node_one, node_two):
    for channel in node_two.channels:
        if channel.node1 == node_one:
            return channel
        if channel.node2 == node_one:
            return channel

chan1 = find_channel(node_one, node_two)
chan2 = find_channel(node_two, node_three)

chan2, chan1

(GossmapChannel[804803x2012x1], GossmapChannel[816483x332x0])

In [56]:
chan1.half_channels[0]

[GossmapHalfchannel[816483x332x0/0], GossmapHalfchannel[816483x332x0/1]]

## Which half-channels (channel update info) are we using for this payment?

In [57]:
def find_direction(source, destination): 
    return 0 if source < destination else 1

In [61]:
direction = find_direction(node_one, node_two)
hc1 = chan1.half_channels[direction]
hc1.source == node_one

True

In [62]:
dir = find_direction(node_two, node_three)
hc2 = chan2.half_channels[dir]
hc2.source == node_two

True

Fields on a halfchannel

        self.timestamp: int 
        self.cltv_expiry_delta: int 
        self.htlc_minimum_msat: int 
        self.htlc_maximum_msat: Optional[int] 
        self.fee_base_msat: int
        self.fee_proportional_millionths: int 
        self.disabled = fields['channel_flags'] & 2 > 0

## Can our payment fit through the route we found?

In [64]:
hc2.disabled and hc1.disabled

False

In [67]:
assert payment_amount < hc2.htlc_maximum_msat
assert payment_amount > hc2.htlc_minimum_msat

In [68]:
assert payment_amount < hc1.htlc_maximum_msat
assert payment_amount > hc1.htlc_minimum_msat

In [70]:
chan1.satoshis > (payment_amount // 1000)

True

In [71]:
chan2.satoshis > (payment_amount // 1000)

True

## How much will it cost us to use this route?

- is in time (cltv deltas)
- is in sats (fee_base_msat + (payment_amount) * proportional)

In [75]:
# What's the minimum timeout i'm going to have to wait?
# - three numbers to add here!
# - two cltv_deltas from each of the channels we're using

final_cltv_from_invoice = 10
hc2.cltv_expiry_delta + final_cltv_from_invoice

154

In [87]:
# how much will we have to pay in fees to route this payment?

half_chans = [hc1, hc2]

def compute_fee(amount_msat, hc):
    fee_msat = hc.fee_base_msat
    fee_msat += hc.fee_proportional_millionths * amount_msat // 1000000
    return fee_msat

fee = compute_fee(payment_amount, half_chans[1])
fee

5000

# 0 -> A -> B -> C

amt_to_send_a = As fees + 10005 -> amt_to_forward 10005 -> amt_to_forward 10000

In [102]:
def compute_route_fee(payment_msat, hc_list):
    total_amt = payment_msat
    total_fee = 0
    hc_list = hc_list[1:]
    for hc in hc_list[::-1]:
        fee = compute_fee(total_amt, hc)
        total_fee += fee
        total_amt += fee
    return total_fee

total_fee_msat = compute_route_fee(payment_amount, half_chans)

# how much is the total fee, as a percent of original payment (in bips)
total_fee_msat / payment_amount * 10000

5.0

Data we could include in an onion payload

    amt_to_forward: msats (8bytes)
    outgoing_cltv_value: blocks (4bytes)
    short_channel_id: bytes??
    payment_data: {
        payment_secret: 32*byte
        total_msat: msats (8bytes)
    }
    encrypted_recipient_data: ...?
    current_blinding_point: point
    payment_metadata: ...*bytes
    total_amount_msat: msat (8bytes)


Non-final hops:
    - outgoing_cltv_value
    - amt_to_forward
    - short_channel_id 
    
Final hop:
    - outgoing_cltv_value
    - amt_to_forward
    - payment_data { payment_secret, total_msat }

In [111]:
# a route is a list of hops [hop, hop, hop]

def make_route(payment_secret, payment_amt, blockheight, initial_cltv_delta, halfchans):
    cltv_blockheight = blockheight
    amt_to_forward = payment_amt

    last_half_chan = halfchans[-1]
    last_hop = {
        'pubkey': str(last_half_chan.destination.node_id),
        'amt_to_forward': amt_to_forward,
        'outgoing_cltv_value': cltv_blockheight,
        'payment_data': {
            'payment_secret': payment_secret, 
            'total_msat': payment_amt, 
        }
    }

    cltv_blockheight += initial_cltv_delta
    route = [last_hop]
    for hc in halfchans[::-1]:
        hop = {
            'pubkey': str(hc.source.node_id),
            'amt_to_forward': amt_to_forward,
            'short_channel_id': hc.channel.scid,
            'outgoing_cltv_value': cltv_blockheight,
        }
        amt_to_forward += compute_fee(amt_to_forward, hc)
        cltv_blockheight += hc.cltv_expiry_delta
        route = [hop] + route
        
    return route

payment_secret = '22' * 32
blockheight = 1000
initial_cltv_delta = 40

route = make_route(payment_secret, payment_amount, blockheight, initial_cltv_delta, half_chans)

In [112]:
route

[{'pubkey': '0235acd5e943cbe5387815a8fb15ebc109b26556f82fa28cfcb0c2a3fda7da80fa',
  'amt_to_forward': 10005000,
  'short_channel_id': ShortChannelId[816483x332x0],
  'outgoing_cltv_value': 1184},
 {'pubkey': '02f1a8c87607f415c8f22c00593002775941dea48869ce23096af27b0cfdcc0b69',
  'amt_to_forward': 10000000,
  'short_channel_id': ShortChannelId[804803x2012x1],
  'outgoing_cltv_value': 1040},
 {'pubkey': '02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26',
  'amt_to_forward': 10000000,
  'outgoing_cltv_value': 1000,
  'payment_data': {'payment_secret': '2222222222222222222222222222222222222222222222222222222222222222',
   'total_msat': 10000000}}]

Now that we've got all the info we need to build the onion, let's make an onion!

And then we'll be ready to send a payment!


```python
    # step three, make an onion
    first_hop = route[0]
    blockheight = plugin.rpc.getinfo()['blockheight']
    hops = []
    for hop, nexthop in zip(route[:-1], route[1:]):
        hops.append({
            'pubkey': hop['id'],
            'payload': serialize_payload_tlv(blockheight, nexthop)
        })

    # The last hop has a special payload:
    hops.append({
        "pubkey": route[-1]['id'],
        "payload": serialize_payload_final_tlv(blockheight, route[-1], invoice['payment_secret'])
    })

    onion = plugin.rpc.createonion(hops=hops, assocdata=invoice['payment_secret'])
```

In [115]:
from pyln.proto.onion import TlvPayload
from io import BytesIO
import struct


def truncate_encode(i: int):
    """Encode a tu64 (or tu32 etc) value"""
    ret = struct.pack("!Q", i)
    while ret.startswith(b'\0'):
        ret = ret[1:]
    return ret


def serialize_payload_tlv(hop):
    block, tx, out = str(hop['short_channel_id']).split('x')

    payload = TlvPayload()
    # BOLT #4:
    #     1. type: 2 (`amt_to_forward`)
    #     2. data:
    #         * [`tu64`:`amt_to_forward`]
    b = BytesIO()
    b.write(truncate_encode(int(hop['amt_to_forward'])))
    payload.add_field(2, b.getvalue())
    # BOLT #4:
    #    1. type: 4 (`outgoing_cltv_value`)
    #    2. data:
    #        * [`tu32`:`outgoing_cltv_value`]
    b = BytesIO()
    b.write(truncate_encode(hop['outgoing_cltv_value']))
    payload.add_field(4, b.getvalue())
    # BOLT #4:
    #    1. type: 6 (`short_channel_id`)
    #    2. data:
    #        * [`short_channel_id`:`short_channel_id`]
    b = BytesIO()
    b.write(struct.pack("!Q", int(block) << 40 | int(tx) << 16 | int(out)))
    payload.add_field(6, b.getvalue())
    return payload.to_bytes().hex()

In [116]:
def serialize_payload_final_tlv(hop):
    payload = TlvPayload()
    # BOLT #4:
    #     1. type: 2 (`amt_to_forward`)
    #     2. data:
    #         * [`tu64`:`amt_to_forward`]
    b = BytesIO()
    b.write(truncate_encode(int(hop['amt_to_forward'])))
    payload.add_field(2, b.getvalue())
    # BOLT #4:
    #    1. type: 4 (`outgoing_cltv_value`)
    #    2. data:
    #        * [`tu32`:`outgoing_cltv_value`]
    b = BytesIO()
    b.write(truncate_encode(hop['outgoing_cltv_value']))
    payload.add_field(4, b.getvalue())
    # BOLT #4:
    #    1. type: 8 (`payment_data`)
    #    2. data:
    #        * [`32*byte`:`payment_secret`]
    #        * [`tu64`:`total_msat`]
    b = BytesIO()
    b.write(bytes.fromhex(hop['payment_data']['payment_secret']))
    b.write(truncate_encode(int(hop['payment_data']['total_msat'])))
    payload.add_field(8, b.getvalue())
    return payload.to_bytes().hex()


In [134]:
onion_hops = []
for hop, next_hop in zip(route[:-1], route[1:]):
    if 'short_channel_id' in next_hop:
        onion_hop = { 
            'pubkey': next_hop['pubkey'],
            'payload': serialize_payload_tlv(next_hop),
        }
    else:
        onion_hop = {
            'pubkey': next_hop['pubkey'],
            'payload': serialize_payload_final_tlv(next_hop)
        }
    onion_hops.append(onion_hop)

onion_hops

[{'pubkey': '02f1a8c87607f415c8f22c00593002775941dea48869ce23096af27b0cfdcc0b69',
  'payload': '1302039896800402041006080c47c30007dc0001'},
 {'pubkey': '02c9477078f21e6f17816257132678d2d27f4c7074c8b0114d8be5ce9cee14bc26',
  'payload': '2e0203989680040203e808232222222222222222222222222222222222222222222222222222222222222222989680'}]

In [135]:
from pyln.client import LightningRpc

rpc = LightningRpc('/tmp/l1-regtest/regtest/lightning-rpc')

In [136]:
onion = rpc.createonion(hops=onion_hops, assocdata=payment_secret)

In [137]:
onion

{'onion': '0003c525b25ef212c8a7ebe847fbd7efb5e8b428c2a24b50886971c206a4e15d656265a2939a51a98d5422cf3694154f725b29d69a2b629b52dd6bc99bbb425e15918634101d33975af0d29fff814329ce28dec051501efd8c6a5ad0ba42ed811784c1b6f80c396163ea675a9fb00079edf60dc0e9af2da64e625c0339dcd76ad29fad80098b8bc1537d46f9c3d102b43baf0865e4a874c88241719356f9a279c1601c0b62753a00e7e86f91f1cf0d901c1aec0d76dc91d285ba9036d038bd4020f9a41f2a85c24b26a00a9a0732fc845af3b36c6e81bcc8a282ed7d77d0372f05cfc545434c126a9e14a8dfa8ac711b470bbabeeaa0c6ec1dcac5e3c6e22f9efd8b5fb40baea1b6d5a81e39d14cd1d769e8d9171d6ba6dbafd885b0d0c6e6b43c10677b72549918059c6474988a12a8ed3b76fbbe1c6147877674a4c3a838b0d88d839fc6a4c3a888cc0894ecb9feaec792042bd8708f579b2076b8007e2ace51bbf3feec26d3bdb5b9c7218499813053ff9effade08cf0aa00349b8e0c9ddc429c37bd6d4c53adb7153b7735883c5d867ee16f509cd9a9bf8648b07b89ea3a9c38df2aa20a11c2d5deb7cdc469f75a65c44b11b7d656a55fabbf9038b6f2ac45e35a84be7b95c7e0c924bd5e5d5c5a190352df480082fa24d78a0fbc3535e420ff1f9cfa460698ed8114ed98248

In [131]:
len(onion['onion']) // 2 - (2 * 33)

1300

In [144]:
payment_hash = '00' * 32

"""
HAVE in `route`
{'pubkey': '0235acd5e943cbe5387815a8fb15ebc109b26556f82fa28cfcb0c2a3fda7da80fa',
  'amt_to_forward': 10005000,
  'short_channel_id': ShortChannelId[816483x332x0],
  'outgoing_cltv_value': 1184},

EXPECT: in `first_hop`
•  id (pubkey): The node at the end of this hop
•  channel (short_channel_id): The channel joining these nodes
•  direction (u32): 0 if this channel is traversed from lesser to greater id, otherwise 1
•  amount_msat (msat): The amount expected by the node at the end of this hop
•  delay (u32): The total CLTV expected by the node at the end of this hop
•  style (string): The features understood by the destination node (always "tlv")
"""
first_hop = {
    'id': route[1]['pubkey'],
    'channel': str(route[0]['short_channel_id']),
    'amount_msat': route[0]['amt_to_forward'],
    'delay': route[0]['outgoing_cltv_value'],
    'style': 'tlv',
}

try:
    rpc.sendonion(onion=onion['onion'], first_hop=first_hop, payment_hash=payment_hash)
except Exception as err:
    print(err)

RPC call failed: method: sendonion, payload: {'onion': '0003c525b25ef212c8a7ebe847fbd7efb5e8b428c2a24b50886971c206a4e15d656265a2939a51a98d5422cf3694154f725b29d69a2b629b52dd6bc99bbb425e15918634101d33975af0d29fff814329ce28dec051501efd8c6a5ad0ba42ed811784c1b6f80c396163ea675a9fb00079edf60dc0e9af2da64e625c0339dcd76ad29fad80098b8bc1537d46f9c3d102b43baf0865e4a874c88241719356f9a279c1601c0b62753a00e7e86f91f1cf0d901c1aec0d76dc91d285ba9036d038bd4020f9a41f2a85c24b26a00a9a0732fc845af3b36c6e81bcc8a282ed7d77d0372f05cfc545434c126a9e14a8dfa8ac711b470bbabeeaa0c6ec1dcac5e3c6e22f9efd8b5fb40baea1b6d5a81e39d14cd1d769e8d9171d6ba6dbafd885b0d0c6e6b43c10677b72549918059c6474988a12a8ed3b76fbbe1c6147877674a4c3a838b0d88d839fc6a4c3a888cc0894ecb9feaec792042bd8708f579b2076b8007e2ace51bbf3feec26d3bdb5b9c7218499813053ff9effade08cf0aa00349b8e0c9ddc429c37bd6d4c53adb7153b7735883c5d867ee16f509cd9a9bf8648b07b89ea3a9c38df2aa20a11c2d5deb7cdc469f75a65c44b11b7d656a55fabbf9038b6f2ac45e35a84be7b95c7e0c924bd5e5d5c5a190352df480082fa